# 作成機能に追加するもの

In [2]:
%%capture
!pip install qdrant-client
!pip install langchain
!pip install openai
!pip install tiktoken

In [10]:
from langchain.embeddings.openai import OpenAIEmbeddings
from qdrant_client import QdrantClient
from qdrant_client import models
import re

## Qdrantの再作成

In [24]:
!rm -r ../tools
path = '../tools'
qdrant = QdrantClient(path=path)

qdrant.create_collection(
    collection_name = 'tool_store',
    vectors_config=models.VectorParams(
        size=1536, # Vector size is defined by used model
        distance=models.Distance.COSINE
        )
)

True

In [14]:
model = 'text-embedding-ada-002'
embeddings = OpenAIEmbeddings(model = model)

In [43]:
# Get the current number of data and add 1
count = qdrant.count(collection_name="tool_store", exact=True)
idx = int(f'{count}'.replace('count=', '')) + 1
print(idx)


discription = "used for calculating the sum of prime numbers from A to B. The input is a single string with two numbers separated by a comma. For example, if you want to find the sum of prime numbers between 1 and 10, the input is '1,10'."

code = """
from langchain.tools import BaseTool

class PrimeSumTool(BaseTool):
    name = "PrimeSumTool"
    description = "used for calculating the sum of prime numbers from A to B. The input is a single string with two numbers separated by a comma. For example, if you want to find the sum of prime numbers between 1 and 10, the input is '1,10'."

    def is_prime(self, num: int) -> bool:
        if num < 2:
            return False
        for i in range(2, num):
            if num % i == 0:
                return False
        return True

    def _run(self, query: str) -> str:
        "Use the tool."
        a, b = query.split(",")
        a, b = int(a), int(b)
        prime_sum = sum([x for x in range(a, b + 1) if self.is_prime(x)])
        return str(prime_sum)

    async def _arun(self, query: str) -> str:
        "Use the tool asynchronously."
        raise NotImplementedError("PrimeSumTool does not support async")
"""
tool_name = re.search(r'name = "(.*?)"', code).group(1)
description = re.search(r'description = "(.*?)"', code).group(1)

payload = {'discription': discription, 'code': code, 'tool_name': tool_name}

embedding = embeddings.embed_query(discription)

# upload data
qdrant.upload_records(
    collection_name="tool_store",
    records=[models.Record(id=idx, vector=embedding, payload=payload)]
)

qdrant.count(collection_name="tool_store", exact=True)

2


CountResult(count=2)

In [44]:
query = "used for calculating the sum of prime numbers from A to B"

search_result = qdrant.search(
    collection_name="tool_store",
    query_vector = embeddings.embed_documents(query)[0],
    limit=3
)

for hit in search_result:
  print(hit.payload)

{'discription': "used for calculating the sum of prime numbers from A to B. The input is a single string with two numbers separated by a comma. For example, if you want to find the sum of prime numbers between 1 and 10, the input is '1,10'.", 'code': '\nfrom langchain.tools import BaseTool\n\nclass PrimeSumTool(BaseTool):\n    name = "PrimeSumTool"\n    description = "used for calculating the sum of prime numbers from A to B. The input is a single string with two numbers separated by a comma. For example, if you want to find the sum of prime numbers between 1 and 10, the input is \'1,10\'."\n\n    def is_prime(self, num: int) -> bool:\n        if num < 2:\n            return False\n        for i in range(2, num):\n            if num % i == 0:\n                return False\n        return True\n\n    def _run(self, query: str) -> str:\n        "Use the tool."\n        a, b = query.split(",")\n        a, b = int(a), int(b)\n        prime_sum = sum([x for x in range(a, b + 1) if self.is_p

In [45]:
tools = []

for i in range(len(search_result)):
    disc = search_result[i].payload['discription']
    code = search_result[i].payload['code']
    tool_name = search_result[i].payload['tool_name']

    tool_code = code + '\n' + f'{tool_name} = {tool_name}()'
    exec(tool_code, globals())
    exec(f'tools.append({tool_name})', globals())

print(tools)
    

[PrimeSumTool(name='PrimeSumTool', description="used for calculating the sum of prime numbers from A to B. The input is a single string with two numbers separated by a comma. For example, if you want to find the sum of prime numbers between 1 and 10, the input is '1,10'.", args_schema=None, return_direct=False, verbose=False, callbacks=None, callback_manager=None, handle_tool_error=False), PrimeSumTool(name='PrimeSumTool', description="used for calculating the sum of prime numbers from A to B. The input is a single string with two numbers separated by a comma. For example, if you want to find the sum of prime numbers between 1 and 10, the input is '1,10'.", args_schema=None, return_direct=False, verbose=False, callbacks=None, callback_manager=None, handle_tool_error=False)]


In [46]:
from langchain.llms import OpenAI
from langchain.agents import initialize_agent

model = OpenAI(temperature=0,model_name = 'gpt-4')
input = 'Sum of prime numbers from 1 to 50'

agent = initialize_agent(tools, model, agent="zero-shot-react-description", verbose=True)

agent.run(input)



> Entering new AgentExecutor chain...
I need to use the PrimeSumTool to find the sum of prime numbers between 1 and 50.
Action: PrimeSumTool
Action Input: 1,50
Observation: 328
Thought:I now know the final answer.
Final Answer: 328

> Finished chain.


'328'